1. 首先是检测SignificantLayer是否存在
2. 接下来统一SignificantLayer里面数据的格式，统一为mhd格式。
3. 然后将所有的Mask文件填充
4. 根据Mask得到Expand_Mask
5. 我们需要对所有的mhd文件统一一下格式，全部统一为三维的1*512*512

In [1]:
import os
import numpy as np
from glob import glob
import SimpleITK as itk
import pydicom
import cv2
PHASE_NAMES = ['NC', 'ART', 'PV']

In [2]:
'''
    此ｂｌｏｃｋ封装一些常见的操作
'''
# 读取单个DICOM文件
def read_dicom_file(file_name):
    header = pydicom.read_file(file_name)
    image = header.pixel_array
    image = header.RescaleSlope * image + header.RescaleIntercept
    return image
# 读取mhd文件
def read_mhd_image(file_path):
    header = itk.ReadImage(file_path)
    image = itk.GetArrayFromImage(header)
    return np.array(image)


# 保存mhd文件
def save_mhd_image(image, file_name):
    header = itk.GetImageFromArray(image)
    itk.WriteImage(header, file_name)
    
def image_erode(image, size):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
    image = cv2.erode(image, kernel)
    return image


def image_expand(image, size):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
    image = cv2.dilate(image, kernel)
    return image

In [3]:
def exists_SignificantLayer(level_dir, levels=['3', '4', '5']):
    '''
        检查是否每个样本都存在'SignificantLayer'文件夹，如果不存在，则输出提醒信息
        如果没有任何信息输出，说明SignificantLayer都存在
    '''
    for level in levels:
        cur_dir = os.path.join(level_dir, level)
        names = os.listdir(cur_dir)
        paths = [os.path.join(cur_dir, name) for name in names]
        for path in paths:
            if not os.path.exists(os.path.join(path, 'SignificantLayer')):
                print 'not exists', path

In [4]:
exists_SignificantLayer(
    '/home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level',
    levels=['1', '2', '3', '4', '5']
)

In [5]:
def unit_format(level_dir, levels=['3', '4', '5']):
    '''
        统一SignificantLayer里面数据的格式，统一为mhd格式。
        如果存在Ｄｉｃｏｍ格式的文件，那么我们需要做下面两部操作
            １、将Ｄｉｃｏｍ文件转化为ｍｈｄ文件,命名方式是: phasename_Image.mhd
            ２、将原来的ｍａｓｋ文件重新命名,命名方式是: phasename_Mask.mhd
    '''
    PHASE_NAMES = ['NC', 'ART', 'PV']
    def exists_DICOM(dir_name):
        names = os.listdir(dir_name)
        for name in names:
            if name.endswith('.DCM'):
                return True
        return False
    def find_proper_maskmhd(sl_dir, phase_name):
        mask_paths = glob(os.path.join(sl_dir, phase_name+'*.mhd'))
        last_path = None
        for mask_path in mask_paths:
            if mask_path.endswith('_Image.mhd'):
                continue
            if mask_path.endswith('_Mask.mhd'):
                return mask_path
            last_path = mask_path
        return last_path
    for level in levels:
        cur_dir = os.path.join(level_dir, level)
        names = os.listdir(cur_dir)
        paths = [os.path.join(cur_dir, name) for name in names]
        for path in paths:
            sl_dir = os.path.join(path, 'SignificantLayer')
            if exists_DICOM(sl_dir):
                for phase_name in PHASE_NAMES:
                    # 第一步
                    dicom_path = glob(os.path.join(sl_dir, phase_name+'*.DCM'))[0]
                    dicom_image = read_dicom_file(dicom_path)
                    save_mhd_image(dicom_image, os.path.join(sl_dir, phase_name+'_Image.mhd'))
                    # 第二步
                    mask_path = find_proper_maskmhd(sl_dir, phase_name)
                    mask_image = read_mhd_image(mask_path)
                    mask_image[mask_image != 1] = 0  # 把非１元素全都置为０
                    save_mhd_image(mask_image, os.path.join(sl_dir, phase_name+'_Mask.mhd'))
                    
                    print 'finish two step in ', path
            else:
                # 第二步
                for phase_name in PHASE_NAMES:
                    mask_path = find_proper_maskmhd(sl_dir, phase_name)
                    mask_image = read_mhd_image(mask_path)
                    mask_image[mask_image != 1] = 0  # 把非１元素全都置为０
                    save_mhd_image(mask_image, os.path.join(sl_dir, phase_name+'_Mask.mhd'))

                    print 'finish two step in ', path

In [7]:
unit_format(
    '/home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level',
    levels=['1', '2', '3', '4', '5']
)

finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3065890-2379159
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3065890-2379159
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3065890-2379159
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3622154-3224937
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3622154-3224937
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3622154-3224937
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/2956711-2810792
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/2956711-2810792
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/2956711-2810792
finish two step in 

finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3616711-3222746
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/304176-2802027
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/304176-2802027
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/304176-2802027
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3076489-2392776
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3076489-2392776
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3076489-2392776
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/2244557-2387317
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/2244557-2387317
finish two step in  /h

finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3458358-2950016
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3458358-2950016
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3368124-2812323
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3368124-2812323
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3368124-2812323
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3449639-2934524
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3449639-2934524
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3449639-2934524
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/2720042-3215676
finish two step in 

finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/506620-2874907
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3228513-2812124
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3228513-2812124
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3228513-2812124
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/435212-2375097
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/435212-2375097
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/435212-2375097
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3580198-3151190
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3580198-3151190
finish two step in  /ho

finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/2646651-3005757
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3085554-2405513
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3085554-2405513
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3085554-2405513
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/2714545-1938082
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/2714545-1938082
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/2714545-1938082
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3504346-3021112
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3504346-3021112
finish two step in 

...

finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3445163-2948204
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3445163-2948204
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3166432-2525487
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3166432-2525487
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3166432-2525487
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3334383-2745801
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3334383-2745801
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3334383-2745801
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3503112-3026209
finish two step in 

finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3595744-3174849
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3595744-3174849
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3595744-3174849
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/1052412-1963904
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/1052412-1963904
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/1052412-1963904
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3498105-3010128
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3498105-3010128
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3498105-3010128
finish two step in 

finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3269059-2650136
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3269059-2650136
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3365022-2795004
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3365022-2795004
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3365022-2795004
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2813385-2065382
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2813385-2065382
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2813385-2065382
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3189351-2546504
finish two step in 

finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2510720-1714729
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2510720-1714729
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2510720-1714729
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2586849-1799988
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2586849-1799988
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2586849-1799988
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3140444-2500631
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3140444-2500631
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3140444-2500631
finish two step in 

finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3126862-2458412
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3126862-2458412
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3126862-2458412
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3358635-2792688
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3358635-2792688
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3358635-2792688
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2970918-2246507
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2970918-2246507
finish two step in  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2970918-2246507
finish two step in 

In [79]:
# def copy_orginal_Mask(level_dir,  levels=['3', '4', '5'], origin_dir='/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data'):
#     '''
#         只拷贝原始的ｍａｓｋ文件
#     '''
#     import shutil
#     for level in levels:
#         cur_dir = os.path.join(level_dir, level)
#         names = os.listdir(cur_dir)
#         paths = [os.path.join(cur_dir, name) for name in names]
#         for path in paths:
#             pcid = os.path.basename(path)
#             sl_dir = os.path.join(origin_dir, pcid, 'SignificantLayer')
#             for phase_name in PHASE_NAMES:
#                 mask_paths = glob(os.path.join(sl_dir, phase_name+'*.mhd'))
#                 for mask_path in mask_paths:
#                     target_path = os.path.join(path, 'SignificantLayer', os.path.basename(mask_path))
#                     print mask_path, '->', target_path
#                     mask_image = read_mhd_image(mask_path)
#                     save_mhd_image(mask_image, target_path)
#                     # shutil.copyfile(mask_path, target_path)
# copy_orginal_Mask('/home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level')

/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3580198-3151190/SignificantLayer/NC_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3580198-3151190/SignificantLayer/NC_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3580198-3151190/SignificantLayer/NC_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3580198-3151190/SignificantLayer/NC_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3580198-3151190/SignificantLayer/ART_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3580198-3151190/SignificantLayer/ART_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3580198-3151190/SignificantLayer/ART_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3580198-3151190/SignificantLayer/ART_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/origina

/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/2322808-2322808/SignificantLayer/NC_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/2322808-2322808/SignificantLayer/NC_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/2322808-2322808/SignificantLayer/NC_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/2322808-2322808/SignificantLayer/NC_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/2322808-2322808/SignificantLayer/ART_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/2322808-2322808/SignificantLayer/ART_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/2322808-2322808/SignificantLayer/ART_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/2322808-2322808/SignificantLayer/ART_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/origina

/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3484536-2992567/SignificantLayer/ART_3_2.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3484536-2992567/SignificantLayer/ART_3_2.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3484536-2992567/SignificantLayer/PV_3_2.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3484536-2992567/SignificantLayer/PV_3_2.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/1018919-2653035/SignificantLayer/NC_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/1018919-2653035/SignificantLayer/NC_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/1018919-2653035/SignificantLayer/NC_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/1018919-2653035/SignificantLayer/NC_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/1

/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3370534-2827401/SignificantLayer/ART_3_7.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3370534-2827401/SignificantLayer/ART_3_7.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3370534-2827401/SignificantLayer/PV_3_7.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3370534-2827401/SignificantLayer/PV_3_7.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3403292-3059121/SignificantLayer/NC_3_3.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3403292-3059121/SignificantLayer/NC_3_3.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3403292-3059121/SignificantLayer/ART_3_3.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3403292-3059121/SignificantLayer/ART_3_3.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/34032

/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3329181-2743396/SignificantLayer/PV_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3329181-2743396/SignificantLayer/PV_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3168385-2514208/SignificantLayer/NC_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3168385-2514208/SignificantLayer/NC_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3168385-2514208/SignificantLayer/NC_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3168385-2514208/SignificantLayer/NC_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3168385-2514208/SignificantLayer/ART_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3168385-2514208/SignificantLayer/ART_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/origina

...

/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3762947-3422066/SignificantLayer/ART_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3762947-3422066/SignificantLayer/ART_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3762947-3422066/SignificantLayer/ART_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3762947-3422066/SignificantLayer/ART_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3762947-3422066/SignificantLayer/PV_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3762947-3422066/SignificantLayer/PV_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3762947-3422066/SignificantLayer/PV_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3762947-3422066/SignificantLayer/PV_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/origina

/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3126862-2458412/SignificantLayer/ART_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3126862-2458412/SignificantLayer/ART_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3126862-2458412/SignificantLayer/PV_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3126862-2458412/SignificantLayer/PV_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3126862-2458412/SignificantLayer/PV_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3126862-2458412/SignificantLayer/PV_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3358635-2792688/SignificantLayer/NC_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3358635-2792688/SignificantLayer/NC_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_

/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/2810769-2050257/SignificantLayer/ART_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2810769-2050257/SignificantLayer/ART_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/2810769-2050257/SignificantLayer/ART_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2810769-2050257/SignificantLayer/ART_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/2810769-2050257/SignificantLayer/PV_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2810769-2050257/SignificantLayer/PV_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/2810769-2050257/SignificantLayer/PV_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2810769-2050257/SignificantLayer/PV_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/origina

/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3048015-2356871/SignificantLayer/NC_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3048015-2356871/SignificantLayer/NC_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3048015-2356871/SignificantLayer/NC_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3048015-2356871/SignificantLayer/NC_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3048015-2356871/SignificantLayer/ART_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3048015-2356871/SignificantLayer/ART_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3048015-2356871/SignificantLayer/ART_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3048015-2356871/SignificantLayer/ART_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/origina

/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3913384-3623220/SignificantLayer/NC_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3913384-3623220/SignificantLayer/NC_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3913384-3623220/SignificantLayer/ART_Image.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3913384-3623220/SignificantLayer/ART_Image.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3913384-3623220/SignificantLayer/ART_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3913384-3623220/SignificantLayer/ART_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_data/3913384-3623220/SignificantLayer/PV_Mask.mhd -> /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3913384-3623220/SignificantLayer/PV_Mask.mhd
/home/give/Documents/dataset/LI-RADS/data/whole_data/original_

In [9]:
def fill_mask_image(level_dir,  levels=['3', '4', '5']):
    '''
        因为有些Ｍａｓｋ文件只画出了边缘，所以我们需要对内部进行填充
    '''
    def fill_one_image(image):
        from scipy import ndimage
        image = ndimage.binary_fill_holes(image).astype(np.uint8)
        return image
    for level in levels:
        cur_dir = os.path.join(level_dir, level)
        names = os.listdir(cur_dir)
        paths = [os.path.join(cur_dir, name) for name in names]
        for path in paths:
            sl_dir = os.path.join(path, 'SignificantLayer')
            print 'check ', sl_dir
            for phase_name in PHASE_NAMES:
                mask_path = os.path.join(sl_dir, phase_name+'_Mask.mhd')
                mask_image = read_mhd_image(mask_path)
                shape = list(np.shape(mask_image))
                if len(shape) == 3:
                    mask_image = mask_image[0, :, :]
                print 'before ',np.shape(mask_image), np.sum(mask_image == 1)
                mask_image = fill_one_image(mask_image)
                print 'after ', np.sum(mask_image == 1)
                if np.sum(mask_image == 1) >= (512 * 512):
                    print 'Error ', mask_path
                    continue
                mask_image = np.expand_dims(mask_image, axis=0)
                save_mhd_image(mask_image, mask_path)

In [10]:
fill_mask_image(
    '/home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level',
    levels=['1', '2', '3', '4', '5']
)

check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3065890-2379159/SignificantLayer
before  (512, 512) 418
after  418
before  (512, 512) 459
after  459
before  (512, 512) 374
after  374
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3622154-3224937/SignificantLayer
before  (512, 512) 90
after  325
before  (512, 512) 80
after  341
before  (512, 512) 88
after  367
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/2956711-2810792/SignificantLayer
before  (512, 512) 382
after  382
before  (512, 512) 361
after  361
before  (512, 512) 269
after  269
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/2857450-2784382/SignificantLayer
before  (512, 512) 947
after  947
before  (512, 512) 1309
after  1309
before  (512, 512) 1280
after  1280
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/1966940-2487518/SignificantLayer
before  (512, 512) 351
aft

after  13567
before  (512, 512) 14172
after  14172
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3285098-2809311/SignificantLayer
before  (512, 512) 1691
after  1691
before  (512, 512) 1713
after  1713
before  (512, 512) 1718
after  1718
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3353017-2859533/SignificantLayer
before  (512, 512) 612
after  612
before  (512, 512) 629
after  629
before  (512, 512) 645
after  645
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3213296-2788937/SignificantLayer
before  (512, 512) 270
after  270
before  (512, 512) 315
after  315
before  (512, 512) 308
after  308
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/1132414-2368465/SignificantLayer
before  (512, 512) 706
after  706
before  (512, 512) 709
after  709
before  (512, 512) 742
after  742
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/309

before  (512, 512) 99
after  459
before  (512, 512) 116
after  629
before  (512, 512) 98
after  509
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3228513-2812124/SignificantLayer
before  (512, 512) 98
after  902
before  (512, 512) 109
after  1090
before  (512, 512) 109
after  1037
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/435212-2375097/SignificantLayer
before  (512, 512) 9767
after  9767
before  (512, 512) 9139
after  9139
before  (512, 512) 9401
after  9401
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3580198-3151190/SignificantLayer
before  (512, 512) 164
after  164
before  (512, 512) 128
after  128
before  (512, 512) 170
after  170
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3382520-2823271/SignificantLayer
before  (512, 512) 770
after  770
before  (512, 512) 578
after  578
before  (512, 512) 791
after  791
check  /home/give/Documents/datase

after  13685
before  (512, 512) 14120
after  14120
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3060906-2372564/SignificantLayer
before  (512, 512) 4127
after  4127
before  (512, 512) 4151
after  4151
before  (512, 512) 3650
after  3650
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3265726-2653881/SignificantLayer
before  (512, 512) 527
after  527
before  (512, 512) 392
after  392
before  (512, 512) 266
after  266
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3157679-2722792/SignificantLayer
before  (512, 512) 784
after  784
before  (512, 512) 1249
after  1249
before  (512, 512) 1051
after  1051
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3128461-2494822/SignificantLayer
before  (512, 512) 1392
after  1392
before  (512, 512) 2044
after  2044
before  (512, 512) 965
after  965
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_lev

after  2426
before  (512, 512) 3533
after  3533
before  (512, 512) 3842
after  3842
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3154041-2923547/SignificantLayer
before  (512, 512) 1586
after  1586
before  (512, 512) 1621
after  1621
before  (512, 512) 1781
after  1781
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3565759-3120933/SignificantLayer
before  (512, 512) 195
after  195
before  (512, 512) 219
after  219
before  (512, 512) 335
after  335
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3445163-2948204/SignificantLayer
before  (512, 512) 711
after  711
before  (512, 512) 926
after  926
before  (512, 512) 876
after  876
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3166432-2525487/SignificantLayer
before  (512, 512) 541
after  541
before  (512, 512) 446
after  446
before  (512, 512) 984
after  984
check  /home/give/Documents/dataset/LI-RADS/data/w

after  414
before  (512, 512) 412
after  412
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2923962-2292991/SignificantLayer
before  (512, 512) 823
after  823
before  (512, 512) 716
after  716
before  (512, 512) 618
after  618
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3377325-2814823/SignificantLayer
before  (512, 512) 3831
after  3831
before  (512, 512) 4207
after  4207
before  (512, 512) 4397
after  4397
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2182184-2773189/SignificantLayer
before  (512, 512) 1291
after  1291
before  (512, 512) 915
after  915
before  (512, 512) 1107
after  1107
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3275090-2657144/SignificantLayer
before  (512, 512) 5358
after  5358
before  (512, 512) 5416
after  5416
before  (512, 512) 5168
after  5168
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5

after  3293
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2829179-2073581/SignificantLayer
before  (512, 512) 1218
after  1218
before  (512, 512) 1436
after  1436
before  (512, 512) 1090
after  1090
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3440190-2932737/SignificantLayer
before  (512, 512) 1506
after  1506
before  (512, 512) 1331
after  1331
before  (512, 512) 1455
after  1455
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3803530-3470553/SignificantLayer
before  (512, 512) 1970
after  1970
before  (512, 512) 1765
after  1765
before  (512, 512) 1928
after  1928
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3414926-2873505/SignificantLayer
before  (512, 512) 465
after  465
before  (512, 512) 507
after  507
before  (512, 512) 11801
after  11801
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2767736-1997987/Significan

after  3438
before  (512, 512) 3566
after  3566
before  (512, 512) 4023
after  4023
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2803559-2043272/SignificantLayer
before  (512, 512) 3823
after  3823
before  (512, 512) 4326
after  4326
before  (512, 512) 4402
after  4402
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3826689-3506892/SignificantLayer
before  (512, 512) 560
after  560
before  (512, 512) 509
after  509
before  (512, 512) 506
after  506
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3677388-3306990/SignificantLayer
before  (512, 512) 5729
after  5729
before  (512, 512) 5850
after  5850
before  (512, 512) 6452
after  6452
check  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3126862-2458412/SignificantLayer
before  (512, 512) 643
after  643
before  (512, 512) 14871
after  14871
before  (512, 512) 14687
after  14687
check  /home/give/Documents/dataset/

In [11]:
def save_expand_mask(level_dir,  levels=['3', '4', '5'], expand_size=10):
    '''
        根据上面得到的Mask,我们对其向外膨胀，得到Mask_Expand
    '''
    for level in levels:
        cur_dir = os.path.join(level_dir, level)
        names = os.listdir(cur_dir)
        paths = [os.path.join(cur_dir, name) for name in names]
        for path in paths:
            sl_dir = os.path.join(path, 'SignificantLayer')
            print 'expand ', sl_dir
            for phase_name in PHASE_NAMES:
                mask_path = os.path.join(sl_dir, phase_name+'_Mask.mhd')
                mask_image = read_mhd_image(mask_path)[0]
                expand_mask_image = image_expand(mask_image, expand_size)
                expand_mask_image = np.expand_dims(expand_mask_image, axis=0)
                save_mhd_image(expand_mask_image, os.path.join(sl_dir, phase_name+'_Mask_Expand.mhd'))
                

In [12]:
save_expand_mask(
    '/home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level',
    levels=['1', '2', '3', '4', '5']
)

expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3065890-2379159/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3622154-3224937/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/2956711-2810792/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/2857450-2784382/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/1966940-2487518/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3462780-3160156/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3356280-2864468/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/1725960-2701744/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/8082242

expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/2778650-2072025/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/1695645-3221304/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/667103-2673186/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3470211-3035142/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3513500-3032386/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3449037-2949060/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/3484536-2992567/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/1018919-2653035/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/3/8126355-

expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3263064-2641652/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2962499-3208997/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3596887-3192773/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/8080203-2831955/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3444239-2921750/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3283611-2673231/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3094192-2423940/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3595744-3174849/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/1052412

expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2276117-2561840/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/8029473-2471728/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3619302-3230059/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2773889-2006675/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2953987-2227478/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3327701-2790374/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3036769-2337412/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2714557-1976998/SignificantLayer
expand  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3514373

In [13]:
def expand3dim(level_dir, levels=['3', '4', '5']):
    '''
        将所有的ｍｈｄ文件统一到三维格式，　１*512*512
    '''
    for level in levels:
        cur_dir = os.path.join(level_dir, level)
        names = os.listdir(cur_dir)
        paths = [os.path.join(cur_dir, name) for name in names]
        for path in paths:
            sl_dir = os.path.join(path, 'SignificantLayer')
            print 'expand3dim ', sl_dir
            paths = glob(os.path.join(sl_dir, '*.mhd'))
            for path in paths:
                image = read_mhd_image(path)
                shape = list(np.shape(image))
                if len(shape) == 2:
                    image = np.expand_dims(image, axis=0)
                save_mhd_image(image, path)

In [15]:
expand3dim(
    '/home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level',
    levels=['1', '2', '3', '4', '5']
)

expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3065890-2379159/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3622154-3224937/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/2956711-2810792/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/2857450-2784382/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/1966940-2487518/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3462780-3160156/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/3356280-2864468/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/1/1725960-2701744/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/wh

expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3375452-2815260/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3375468-2824331/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/3051462-2356703/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/911472-3152747/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/1828646-2788204/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/2338414-2784760/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/1440700-3039729/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/2/2170999-3237651/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/who

expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3364727-2795009/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/2760480-1997304/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/4038252-3801374/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/2145516-2772486/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3245102-2750207/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/2450847-1652689/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3595077-3175407/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/4/3361967-2790988/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/wh

expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/8159338-3202896/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3247070-2620063/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2724150-1944611/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2542235-1747625/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3643525-3258636/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2965287-2809549/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3504418-30182002/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3332654-2745818/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/w

expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2965287-2239225/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3668105-3293461/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3762947-3422066/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3347183-2766992/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2767306-1999418/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/2803559-2043272/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3826689-3506892/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/whole_data/category_by_level/5/3677388-3306990/SignificantLayer
expand3dim  /home/give/Documents/dataset/LI-RADS/data/wh